<a href="https://colab.research.google.com/github/SKAbhirama/NLP_RAMKI/blob/main/News-Categorisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [39]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

##Code for removing stopwords

In [ ]:

# Download the NLTK stop words dataset
nltk.download('stopwords')

# Load the Excel file into a pandas DataFrame
excel_path = '/english_news_dataset.xlsx'  # Update with your file path
df = pd.read_excel(excel_path)

# Specify the columns containing the text data
column_name1 = 'Headline'  # Replace with your column name
column_name2 = 'Content'

# Function to remove special characters using regular expressions
def remove_special_characters(text):
    # Replace quotes and hyphens with white spaces
    text = re.sub(r'[\'\"-]', ' ', str(text))
    # Keep only alphanumeric characters and spaces
    pattern = r'[^a-zA-Z0-9\s]'
    return re.sub(pattern, '', text)

# Function to remove stop words
def remove_stop_words(text):
    stop_words = set(stopwords.words('english'))
    words = nltk.word_tokenize(str(text))
    filtered_text = ' '.join([word for word in words if word.lower() not in stop_words])
    return filtered_text

# Apply the functions to the specified columns
df[column_name1] = df[column_name1].apply(remove_special_characters).apply(remove_stop_words)
df[column_name2] = df[column_name2].apply(remove_special_characters).apply(remove_stop_words)

# Save the DataFrame to a new Excel file
output_excel_path = '/content/updated.xlsx'  # Update with your desired output file path
df.to_excel(output_excel_path, index=False)

# Display the modified DataFrame
df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Headline,Content,News Categories,Date
0,Congress leader Baljinder Singh shot dead home...,Congress leader Baljinder Singh shot dead hous...,['national'],19-09-2023
1,17 year old girl preparing NEET dies suicide R...,Another NEET aspirant died suicide Rajasthan K...,['national'],19-09-2023
2,Hampers welcome MPs new Parliament tomorrow pi...,order mark first ever working day new Parliame...,['national'],19-09-2023
3,10 women lawmakers RS 14 LS INC Kharge,Congress President Mallikarjun Kharge speaking...,['national'],19-09-2023
4,Ganesh temple decorated notes coins worth 25 c...,Sri Sathya Ganapathi Temple Bengaluru adorned ...,['national'],19-09-2023


##Code for Lemmatizing the first 10k rows

In [ ]:
!pip install spacy && python -m spacy download en_core_web_sm

In [ ]:
import pandas as pd
import spacy

# Load spaCy's English language model
nlp = spacy.load('en_core_web_sm')

# Load the Excel file into a pandas DataFrame
excel_path = '/content/updated.xlsx'  # Update with your file path
df = pd.read_excel(excel_path)
subset_df = df.head(10000)

# Specify the columns containing the text data
column_name1 = 'Headline'  # Replace with your column name
column_name2 = 'Content'

# Function to apply lemmatization
def lemmatize_text(text):
    doc = nlp(str(text))
    lemmatized_text = ' '.join([token.lemma_ for token in doc])
    return lemmatized_text

# Apply the lemmatization function to the specified columns
subset_df[column_name1] = subset_df[column_name1].apply(lemmatize_text)
subset_df[column_name2] = subset_df[column_name2].apply(lemmatize_text)

# Save the DataFrame to a new Excel file
output_excel_path = '/content/lemmatized_10k.xlsx'  # Update with your desired output file path
subset_df.to_excel(output_excel_path, index=False)

# Display the first 5 rows of the modified DataFrame
subset_df.head()

<ipython-input-4-35a6df35efd5>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df[column_name1] = subset_df[column_name1].apply(lemmatize_text)
<ipython-input-4-35a6df35efd5>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df[column_name2] = subset_df[column_name2].apply(lemmatize_text)


,Headline,Content,News Categories,Date
0,Congress leader Baljinder Singh shoot dead hom...,Congress leader Baljinder Singh shoot dead hou...,['national'],19-09-2023
1,17 year old girl prepare NEET die suicide Raja...,another NEET aspirant die suicide Rajasthan Ko...,['national'],19-09-2023
2,hamper welcome mps new Parliament tomorrow pic...,order mark first ever work day new Parliament ...,['national'],19-09-2023
3,10 woman lawmaker rs 14 LS INC Kharge,Congress President Mallikarjun Kharge speak Ra...,['national'],19-09-2023
4,ganesh temple decorate note coin worth 25 cror...,Sri Sathya Ganapathi Temple Bengaluru adorn pr...,['national'],19-09-2023


In [ ]:
import pandas as pd
import spacy

# Load spaCy's English language model
nlp = spacy.load('en_core_web_sm')

# Load the Excel file into a pandas DataFrame
excel_path = '/content/updated.xlsx'  # Update with your file path
df = pd.read_excel(excel_path)
subset_df = df.head(25000)

# Specify the columns containing the text data
column_name1 = 'Headline'  # Replace with your column name
column_name2 = 'Content'

# Function to apply lemmatization
def lemmatize_text(text):
    doc = nlp(str(text))
    lemmatized_text = ' '.join([token.lemma_ for token in doc])
    return lemmatized_text

# Apply the lemmatization function to the specified columns
subset_df[column_name1] = subset_df[column_name1].apply(lemmatize_text)
subset_df[column_name2] = subset_df[column_name2].apply(lemmatize_text)

# Save the DataFrame to a new Excel file
output_excel_path = '/content/lemmatized_25k.xlsx'  # Update with your desired output file path
subset_df.to_excel(output_excel_path, index=False)

# Display the first 5 rows of the modified DataFrame
subset_df.head()

<ipython-input-17-59717c191f91>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df[column_name1] = subset_df[column_name1].apply(lemmatize_text)
<ipython-input-17-59717c191f91>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df[column_name2] = subset_df[column_name2].apply(lemmatize_text)


,Headline,Content,News Categories,Date
0,Congress leader Baljinder Singh shoot dead hom...,Congress leader Baljinder Singh shoot dead hou...,['national'],19-09-2023
1,17 year old girl prepare NEET die suicide Raja...,another NEET aspirant die suicide Rajasthan Ko...,['national'],19-09-2023
2,hamper welcome mps new Parliament tomorrow pic...,order mark first ever work day new Parliament ...,['national'],19-09-2023
3,10 woman lawmaker rs 14 LS INC Kharge,Congress President Mallikarjun Kharge speak Ra...,['national'],19-09-2023
4,ganesh temple decorate note coin worth 25 cror...,Sri Sathya Ganapathi Temple Bengaluru adorn pr...,['national'],19-09-2023


##Code for creating a TF-IDF matrix from the lemmatized sheet

In [17]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

# Load your lemmatized data from Excel
lemmatized_data_path = '/content/lemmatized_10k.xlsx'
df_lemmatized = pd.read_excel(lemmatized_data_path)

# Load the 'News Categories' data from another Excel sheet
news_categories_path = '/content/updated.xlsx'
df_categories = pd.read_excel(news_categories_path)

# Function to clean 'News Categories' column
def clean_categories(category):
    if isinstance(category, str):
        # Remove non-alphabetic characters and truncate text after a comma
        cleaned_category = ''.join(filter(str.isalpha, category.split(',')[0]))
        return cleaned_category.lower()  # Convert to lowercase for consistency
    return category

# Clean the 'News Categories' column
df_categories['News Categories'] = df_categories['News Categories'].apply(clean_categories)

# Split the data into batches of 4 rows
batch_size = 2500
num_batches = len(df_lemmatized) // batch_size

# Create a TF-IDF vectorizer
vectorizer = TfidfVectorizer(stop_words='english')

# List to store TF-IDF matrices for each batch
tfidf_matrices = []

# Process data in batches
for batch_num in range(num_batches):
    start_idx = batch_num * batch_size
    end_idx = (batch_num + 1) * batch_size

    # Extract the current batch
    current_batch = df_lemmatized['Headline'].iloc[start_idx:end_idx]

    # Fit and transform the current batch
    tfidf_batch = vectorizer.fit_transform(current_batch)

    # Store the TF-IDF matrix in the list
    tfidf_matrices.append(tfidf_batch)

# Concatenate all TF-IDF matrices into one matrix
all_tfidf_matrices = pd.concat([pd.DataFrame(matrix.toarray()) for matrix in tfidf_matrices], ignore_index=True)

# Add the 'News Categories' column to the final DataFrame
all_tfidf_matrices['Category'] = df_categories['News Categories']

vectorizer_path = '/content/vectorizer.joblib'
joblib.dump(vectorizer, vectorizer_path)

# Save the final DataFrame to a CSV file
all_tfidf_matrices.to_csv('/content/final_tfidf_output1.csv', index=False)

all_tfidf_matrices.head()

,0,1,2,3,4,5,6,7,8,9,...,5024,5025,5026,5027,5028,5029,5030,5031,5032,Category
0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,national
1,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,national
2,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,national
3,0.0,0.0,0.289156,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,national
4,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,national


## Word2Vec for alternate usage instead of TF-IDF

In [21]:
import pandas as pd
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split

# Load the Excel file into a pandas DataFrame
excel_path = '/content/lemmatized_10k.xlsx'  # Update with your file path
df = pd.read_excel(excel_path)

# Combine the two columns into a single column (if needed)
# Assuming the two columns are 'Column1' and 'Column2'
# df['Combined'] = df['Column1'] + ' ' + df['Column2']

# Split the data into training and testing sets
X_train, X_test = train_test_split(df['Headline'], test_size=0.2, random_state=42)

# Tokenize the text into words for Word2Vec
tokenized_train_data = [sentence.split() for sentence in X_train]
tokenized_test_data = [sentence.split() for sentence in X_test]

# Train Word2Vec model
word2vec_model = Word2Vec(sentences=tokenized_train_data, vector_size=100, window=5, min_count=1, workers=4)

# Transform the training and testing data into word vectors
word_vectors_train = []
for sentence in tokenized_train_data:
    word_vector = [word2vec_model.wv[word] for word in sentence if word in word2vec_model.wv]
    if word_vector:
        word_vectors_train.append(word_vector)

word_vectors_test = []
for sentence in tokenized_test_data:
    word_vector = [word2vec_model.wv[word] for word in sentence if word in word2vec_model.wv]
    if word_vector:
        word_vectors_test.append(word_vector)

# Convert the word vectors to pandas DataFrames (optional)
wordvec_train_df = pd.DataFrame(word_vectors_train)
wordvec_test_df = pd.DataFrame(word_vectors_test)

# Or save them to a new Excel file
output_excel_path = '/content/wordvec_output.csv'  # Update with your desired output file path
wordvec_train_df.to_csv(output_excel_path, index=False)

model_path = '/content/your_word2vec_model.bin'  # Replace with your desired file path
word2vec_model.save(model_path)

# Display the DataFrames or save them to a new Excel file (optional)
wordvec_train_df.head()
wordvec_test_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,"[-0.081876755, 0.16861151, 0.14033742, -0.0101...","[-0.012668964, 0.031444035, 0.034869775, -0.00...","[-0.032549363, 0.047437374, 0.046893626, 0.004...","[-0.024996918, 0.05508949, 0.033996083, 0.0065...","[-0.0255098, 0.030763188, 0.03203566, 0.001124...","[-0.07856851, 0.15727645, 0.13865736, 0.005178...","[-0.037477337, 0.09919152, 0.07332397, 0.00062...",None,None,None,None,None,None,None,None
1,"[-0.013376156, 0.034494705, 0.029465713, 0.002...","[-0.01234044, 0.039658394, 0.026532164, -0.008...","[-0.049679086, 0.09847899, 0.07852932, 0.00572...","[-0.022389198, 0.043822236, 0.023191858, 0.009...","[-0.018168073, 0.050369114, 0.03852522, 0.0063...","[-0.0425851, 0.108370416, 0.081643745, -0.0043...","[-0.054407336, 0.092695504, 0.075207405, -0.00...","[-0.19651596, 0.39358804, 0.31344932, 0.019508...","[-0.06086446, 0.13051656, 0.1107365, -0.008707...","[-0.05003908, 0.09489324, 0.066053696, 0.01129...",None,None,None,None,None
2,"[-0.03140001, 0.08049798, 0.06268058, 0.009656...","[-0.018534856, 0.0364368, 0.019826172, 0.00365...","[-0.048490718, 0.11181299, 0.08517806, 0.01015...","[-0.009042145, 0.01206482, 0.014906677, 0.0002...","[-0.012569612, 0.037076335, 0.019779805, -0.00...","[-0.066265576, 0.15188657, 0.1080792, -0.00949...","[-0.036267087, 0.083607085, 0.07392618, -0.002...",None,None,None,None,None,None,None,None
3,"[-0.07062672, 0.13063647, 0.10234133, 0.001784...","[-0.19352551, 0.4191492, 0.33881107, 0.0007542...","[-0.16935731, 0.3390619, 0.26106736, -0.001665...","[-0.04860279, 0.10785304, 0.07291446, 0.006002...","[-0.05505233, 0.10704877, 0.082680956, 0.00656...","[-0.056810416, 0.10099567, 0.08910537, 0.00276...","[-0.04913794, 0.11851176, 0.101041175, -0.0041...","[-0.36779523, 0.77885956, 0.62929034, 0.007180...","[-0.2731129, 0.5911776, 0.4877305, -0.02610113...",None,None,None,None,None,None
4,"[-0.16600414, 0.3495721, 0.2658722, -0.0001687...","[-0.20196031, 0.3869458, 0.30395743, -0.024318...","[-0.0052651064, 0.03312764, 0.014551527, -0.00...","[-0.03205229, 0.05062544, 0.031969476, -0.0100...","[-0.050624534, 0.103264526, 0.078072175, -0.00...","[-0.24831761, 0.511496, 0.4240208, -0.00623918...","[-0.023101369, 0.034860812, 0.03984123, -0.000...","[-0.08753711, 0.1913086, 0.16209821, 0.00785, ...","[-0.05892821, 0.119998954, 0.10175443, 0.00028...",None,None,None,None,None,None


###Code to add the 'Category' column

In [22]:
import pandas as pd
from gensim.models import Word2Vec

# Load your Word2Vec matrix CSV
word2vec_matrix_path = '/content/wordvec_output.csv'  # Replace with your actual Word2Vec matrix file path
df_word2vec = pd.read_csv(word2vec_matrix_path)

# Load your 'News Categories' CSV
categories_csv_path = '/content/updated.xlsx'  # Replace with your actual 'News Categories' file path
df_categories = pd.read_excel(categories_csv_path)

# Clean the 'News Categories' column
df_categories['News Categories'] = df_categories['News Categories'].apply(lambda x: x.split(',')[0].replace("[^a-zA-Z]", ""))

# Append the 'News Categories' column to the Word2Vec matrix DataFrame
df_word2vec['Category'] = df_categories['News Categories']

# Save the combined DataFrame to a new CSV file
combined_csv_path = '/content/wordvec_output.csv'  # Replace with your desired combined CSV file path
df_word2vec.to_csv(combined_csv_path, index=False)\

df_word2vec.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,Category
0,[-0.00134385 0.01794615 0.01059241 -0.006493...,[-0.08085521 0.14525616 0.12536278 -0.001819...,[-0.08916104 0.17758545 0.15405993 0.007540...,[-2.11365283e-01 4.52479243e-01 3.52258205e-...,[ 6.33377058e-05 1.04921563e-02 3.63072613e-...,[ 0.00225904 0.02208974 0.01673868 0.005982...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['national']
1,[-0.07577445 0.17390035 0.12980619 -0.006724...,[-1.49840768e-02 1.22462474e-02 8.71939678e-...,[-0.07991815 0.14965466 0.11847727 -0.009966...,[-0.12992227 0.25448972 0.20653965 0.007215...,[-8.58374164e-02 1.64273918e-01 1.20411426e-...,[-0.06394655 0.11731541 0.08417706 -0.006229...,[-0.1950915 0.43689105 0.31501395 -0.023635...,[-0.0028068 0.01731257 0.00836492 -0.007580...,[-5.17344996e-02 9.57802236e-02 6.81773722e-...,[-0.05620351 0.09521054 0.07526133 0.005385...,[-3.37724775e-01 7.02031016e-01 5.11618257e-...,[-0.18911463 0.42525896 0.2869569 0.010884...,NaN,NaN,NaN,['national']
2,[-0.14464907 0.2794025 0.21677819 -0.012041...,[-0.07486121 0.13495569 0.11807755 0.000984...,[-0.15089324 0.3099022 0.2317232 -0.007361...,[-0.01505417 0.03313095 0.02368319 0.004234...,[-8.86668116e-02 2.00143546e-01 1.60612956e-...,[-0.05874008 0.12783955 0.1122807 -0.002990...,[-0.02356208 0.05753254 0.04864805 0.004323...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['national']
3,[-1.5424474e-03 1.4509119e-02 1.6071044e-02 ...,[-0.02213475 0.04273368 0.03648769 0.009918...,[-0.09274183 0.19247712 0.13834627 0.004996...,[ 0.00161786 0.02388408 0.01002667 -0.004300...,[-0.2913632 0.56864846 0.42715934 0.009663...,[-8.16569179e-02 1.57256901e-01 1.25832036e-...,[-7.74608776e-02 1.89704061e-01 1.41621128e-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['national']
4,[-3.37724775e-01 7.02031016e-01 5.11618257e-...,[-3.22807670e-01 6.72000945e-01 4.76466805e-...,[-0.38373575 0.80425245 0.63339776 0.004816...,[-0.1330141 0.233535 0.18480937 -0.010834...,[-0.22500338 0.45924518 0.350693 -0.008279...,[-2.34195665e-01 4.22423273e-01 3.01305264e-...,[-0.02505045 0.02141491 0.02983204 0.008813...,[-2.50557721e-01 5.55008292e-01 4.43929195e-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['national']


##Training and testing the model using HistGradientBoosting Classifier

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn import metrics

# Load the Excel file into a pandas DataFrame
excel_path = '/content/final_tfidf_output.csv'  # Update with your file path
df = pd.read_csv(excel_path)

df_no_nan = df.dropna()

# Assuming the last column is your target labels, and the rest are features
X = df_no_nan.iloc[:, :-1]  # Features (TF-IDF matrix)
y = df_no_nan.iloc[:, -1]   # Target labels

# Split the data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the HistGradientBoostingClassifier
hist_gradient_boosting_classifier = HistGradientBoostingClassifier(random_state=42)
hist_gradient_boosting_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = hist_gradient_boosting_classifier.predict(X_test)

# Evaluate the classifier's performance
accuracy = metrics.accuracy_score(y_test, y_pred)
precision = metrics.precision_score(y_test, y_pred, average='weighted')
recall = metrics.recall_score(y_test, y_pred, average='weighted')
f1_score = metrics.f1_score(y_test, y_pred, average='weighted')

# Display the evaluation metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1_score:.4f}")

##Training and testing the model using SVM Classifiers

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Load the TF-IDF matrix and 'News Categories' from CSV
tfidf_matrix_path = '/content/final_tfidf_output.csv'
df_tfidf = pd.read_csv(tfidf_matrix_path)

# Drop samples with NaN values
df_tfidf_no_nan = df_tfidf.dropna()

# Split the data into features (X) and target labels (y)
X = df_tfidf_no_nan.drop('Category', axis=1)
y = df_tfidf_no_nan['Category']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the SVM classifier
svm_classifier = SVC()

# Train the SVM classifier on the training data
svm_classifier.fit(X_train, y_train)

# Make predictions on the test data
y_pred = svm_classifier.predict(X_test)

# Evaluate the performance of the classifier
accuracy = accuracy_score(y_test, y_pred)
classification_report_str = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report_str)

Accuracy: 0.628
Classification Report:
                 precision    recall  f1-score   support

       asiacup       0.00      0.00      0.00         3
    automobile       1.00      0.71      0.83        24
      business       0.69      0.95      0.80        64
cryptocurrency       0.00      0.00      0.00         1
     education       0.83      0.17      0.29        29
 entertainment       0.71      0.73      0.72        37
       fashion       0.00      0.00      0.00         8
         hatke       0.00      0.00      0.00         2
 healthfitness       0.00      0.00      0.00        10
 miscellaneous       0.56      0.20      0.29        25
      national       0.48      0.77      0.59        57
      politics       0.91      0.95      0.93        56
       science       0.83      0.38      0.53        39
        sports       0.94      0.44      0.60        34
       startup       1.00      0.44      0.62        18
    technology       0.75      0.27      0.40        11
       

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##Training and testing the model using Naive Bayes Classifier

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Load the TF-IDF matrix and 'News Categories' from CSV
tfidf_matrix_path = '/content/final_tfidf_output.csv'
df_tfidf = pd.read_csv(tfidf_matrix_path)

# Drop samples with NaN values
df_tfidf_no_nan = df_tfidf.dropna()

# Split the data into features (X) and target labels (y)
X = df_tfidf_no_nan.drop('Category', axis=1)
y = df_tfidf_no_nan['Category']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Multinomial Naive Bayes classifier
naive_bayes_classifier = MultinomialNB()

# Train the classifier on the training data
naive_bayes_classifier.fit(X_train, y_train)

# Make predictions on the test data
y_pred = naive_bayes_classifier.predict(X_test)

# Evaluate the performance of the classifier
accuracy = accuracy_score(y_test, y_pred)
classification_report_str = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report_str)

Accuracy: 0.554
Classification Report:
                 precision    recall  f1-score   support

       asiacup       0.00      0.00      0.00         3
    automobile       1.00      0.54      0.70        24
      business       0.48      0.91      0.63        64
cryptocurrency       0.00      0.00      0.00         1
     education       1.00      0.07      0.13        29
 entertainment       0.73      0.65      0.69        37
       fashion       0.00      0.00      0.00         8
         hatke       0.00      0.00      0.00         2
 healthfitness       0.00      0.00      0.00        10
 miscellaneous       0.00      0.00      0.00        25
      national       0.45      0.60      0.51        57
      politics       0.57      0.96      0.72        56
       science       0.91      0.26      0.40        39
        sports       0.94      0.44      0.60        34
       startup       0.86      0.33      0.48        18
    technology       1.00      0.09      0.17        11
       

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##Training and testing the model using Random Forest Classifier

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load the TF-IDF matrix and 'News Categories' from CSV
tfidf_matrix_path = '/content/final_tfidf_output.csv'
df_tfidf = pd.read_csv(tfidf_matrix_path)

# Drop samples with NaN values
df_tfidf_no_nan = df_tfidf.dropna()

# Split the data into features (X) and target labels (y)
X = df_tfidf_no_nan.drop('Category', axis=1)
y = df_tfidf_no_nan['Category']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Random Forest classifier
random_forest_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier on the training data
random_forest_classifier.fit(X_train, y_train)

# Make predictions on the test data
y_pred = random_forest_classifier.predict(X_test)

# Evaluate the performance of the classifier
accuracy = accuracy_score(y_test, y_pred)
classification_report_str = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report_str)

''' Load the original vectorizer used during training
vectorizer_path = '/content/vectorizer.joblib'  # Change this to the path where you saved your original vectorizer
vectorizer = joblib.load(vectorizer_path)

def get_tfidf_matrix(input_text, vectorizer):
    # Transform the input text using the TF-IDF vectorizer
    tfidf_matrix = vectorizer.transform(input_text)
    return tfidf_matrix

# New input news sample for prediction
new_input_news = ["Govt redefines 'terrorist act', includes threat to monetary stability"]

# Transform the new input news using the separate method
df_tfidf_new = get_tfidf_matrix(new_input_news, vectorizer)

# Make predictions on the new input news
y_new_pred = random_forest_classifier.predict(df_tfidf_new)

print("Predicted Category for New Input News:", y_new_pred)'''

Accuracy: 0.69
Classification Report:
                 precision    recall  f1-score   support

       asiacup       0.50      0.33      0.40         3
    automobile       0.88      0.88      0.88        24
      business       0.65      0.84      0.73        64
cryptocurrency       1.00      1.00      1.00         1
     education       0.83      0.69      0.75        29
 entertainment       0.60      0.73      0.66        37
       fashion       0.33      0.12      0.18         8
         hatke       0.00      0.00      0.00         2
 healthfitness       0.00      0.00      0.00        10
 miscellaneous       0.37      0.40      0.38        25
      national       0.59      0.61      0.60        57
      politics       0.85      0.95      0.90        56
       science       0.85      0.72      0.78        39
        sports       0.92      0.68      0.78        34
       startup       0.83      0.56      0.67        18
    technology       0.55      0.55      0.55        11
        

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
